In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import numpy as np

In [12]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive


In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = 'cpu'
device

device(type='cuda')

In [4]:
!pip install import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00


## **Load and Prepare Data**

In [5]:
import import_ipynb
import dataset
from dataset import parse_dataset
import os
import pandas as pd

importing Jupyter notebook from dataset.ipynb


In [6]:
labels = [
    'Benign',
    'Bot',
    'Brute Force -XSS',
    'DDOS attack-HOIC',
    'DDOS attack-LOIC-UDP',
    'DDoS attacks-LOIC-HTTP',
    'DoS attacks-GoldenEye',
    'DoS attacks-Hulk',
    'DoS attacks-Slowloris',
    'FTP-BruteForce',
    'SSH-Bruteforce',
    'Label',
    'Brute Force -Web',
    'DoS attacks-SlowHTTPTest',
    'Infilteration',
    'SQL Injection'
]

labels_emb = {}
for i, label in enumerate(labels):
    labels_emb[label] = i

### **Train data classes**
- Benign                      
- Bot                                           
- Brute Force -XSS                 
- DDOS attack-HOIC              
- DDOS attack-LOIC-UDP            
- DDoS attacks-LOIC-HTTP      
- DoS attacks-GoldenEye          
- DoS attacks-Hulk                    
- DoS attacks-Slowloris          
- FTP-BruteForce                                
- Label                                                
- SSH-Bruteforce


In [7]:
train_labels = [
    'Bot',
    'Brute Force -XSS',
    'DDOS attack-HOIC',
    'DDOS attack-LOIC-UDP',
    'DDoS attacks-LOIC-HTTP',
    'DoS attacks-GoldenEye',
    'DoS attacks-Hulk',
    'DoS attacks-Slowloris',
    'FTP-BruteForce',
    'SSH-Bruteforce',
    'Label',

]


### **Test data will consists of classes**
- Brute Force -Web                 
- DoS attacks-SlowHTTPTest      
- Infilteration                                              
- SQL Injection                     


In [8]:
test_labels = [label for label in labels if label not in train_labels]

In [9]:
columns_to_drop = ['Timestamp', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port']

In [15]:
PICKLE_PATH = '../data/pickle/' if not IN_COLAB else '/content/drive/MyDrive/STUDIA/INZ/data/pickle'
TRAIN_PICKLE = 'train_dataset.pkl'
TEST_PICKLE = 'test_dataset.pkl'

In [17]:
train_pickle_path = os.path.join(PICKLE_PATH, TRAIN_PICKLE)
test_pickle_path = os.path.join(PICKLE_PATH, TEST_PICKLE)

if TRAIN_PICKLE not in os.listdir(PICKLE_PATH) and TEST_PICKLE not in os.listdir(PICKLE_PATH):
    ids_dataset = dataset.DatasetIDS2018(csv_file_name='../data/raw/small_merge_data.csv')
    train = ids_dataset.get_data_by_labels(labels=train_labels)
    train_dataset = parse_dataset(dataset=train, columns_to_drop=columns_to_drop, fixed_type=float, labels_emb=labels_emb, labels_column_name='Label')
    train_dataset.to_pickle(train_pickle_path)
    test = ids_dataset.get_data_by_labels(labels=test_labels)
    test_dataset = parse_dataset(dataset=test, columns_to_drop=columns_to_drop, fixed_type=float, labels_emb=labels_emb, labels_column_name='Label')
    test_dataset.to_pickle(test_pickle_path)
else:
    train_dataset = pd.read_pickle(train_pickle_path)
    test_dataset = pd.read_pickle(test_pickle_path)



In [18]:
all_labels = torch.from_numpy(test_dataset.values[:, -1].astype(float)).float()
all_labels

tensor([ 0.,  0.,  0.,  ...,  0., 14., 14.])

In [19]:
len(all_labels)

6414673

In [20]:
test_dataset = data.TensorDataset(torch.from_numpy(test_dataset.values).float(),torch.from_numpy(test_dataset.values[:,-1].astype(float)).float())
train_dataset = data.TensorDataset(torch.from_numpy(train_dataset.values).float(),torch.from_numpy(train_dataset.values[:,-1].astype(float)).float())

In [21]:
len(test_dataset)

6414673

In [22]:
batch_size = 32

In [23]:
print(f'Train len: {len(train_dataset)}\tTrain len % batch_size = {len(train_dataset) % batch_size}\nTest len: {len(test_dataset)}\tTest len % batch_size = {len(test_dataset) % batch_size}')

Train len: 1869581	Train len % batch_size = 13
Test len: 6414673	Test len % batch_size = 17


In [24]:
train_data_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=True)

In [25]:
next(iter(train_data_loader))

[tensor([[6.0000e+00, 1.2883e+04, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          3.0000e+00],
         [6.0000e+00, 4.8300e+02, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          1.0000e+00],
         [6.0000e+00, 1.2561e+05, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          7.0000e+00],
         ...,
         [6.0000e+00, 1.3680e+03, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          3.0000e+00],
         [6.0000e+00, 1.5440e+03, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          3.0000e+00],
         [6.0000e+00, 6.3924e+04, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          7.0000e+00]]),
 tensor([ 3.,  1.,  7.,  3.,  9.,  9.,  7.,  3.,  7.,  3.,  3.,  1.,  1.,  3.,
          3.,  3.,  1.,  1., 10.,  9.,  1.,  1.,  3.,  3.,  1.,  7.,  7.,  7.,
          3.,  3.,  3.,  7.])]

In [26]:
next(iter(test_data_loader))

[tensor([[ 6.0000e+00,  5.3839e+07,  2.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           3.7148e-02,  5.3839e+07,  0.0000e+00,  5.3839e+07,  5.3839e+07,
           5.3839e+07,  5.3839e+07,  0.0000e+00,  5.3839e+07,  5.3839e+07,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.0000e+01,
           0.0000e+00,  3.7148e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  2.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           2.7900e+02, -1

### **Variables**

In [27]:
word_vector_size = 100
input_dim = len(train_dataset[0][0])
output_dim = len(labels)

In [28]:
print(f'Input dim: {input_dim}\nOutput dim: {output_dim}')

Input dim: 78
Output dim: 16


## **Word2Vec**

In [29]:
import string
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

In [30]:
corpus = []
for label in labels:
    words = [word.translate(str.maketrans('', '', string.punctuation)) for word in label.split()]
    corpus.append(words)

In [31]:
gensim_model = Word2Vec(corpus, vector_size=word_vector_size, min_count=1)

In [32]:
class LabelsEmbeddings():
    def __init__(self, gensim_model: Word2Vec):
        self.model = gensim_model

    def fix_vectors_sizes(self, vectors: list) -> list:
        fixed_vectors = []
        max_size = max([len(v) for v in vectors])
        for vector in vectors:
            size_diff = max_size - len(vector)
            vector.extend([[0] * word_vector_size] * size_diff)
            fixed_vectors.append(vector)
        return fixed_vectors

    def generate_vectors(self, labels: dict):
        vectors = []
        for label in labels:
            description_vector = []
            words = [word.translate(str.maketrans('', '', string.punctuation)) for word in label.split()]
            for word in words:
                if word in self.model.wv.index_to_key:
                    description_vector.append(self.model.wv[word])
                else:
                    description_vector.append(word_vector_size * [0])
            vectors.append(description_vector)
        return self.fix_vectors_sizes(vectors)

In [33]:
labels_embeddings = LabelsEmbeddings(gensim_model=gensim_model)
labels_vectors = labels_embeddings.generate_vectors(labels=labels)

In [34]:
[len(v) for v in labels_vectors]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [35]:
for vector in labels_vectors:
    for word in vector:
        assert len(word) == word_vector_size


### **Map layer**
Last layer of the model should be a map between incident embeddings and labels embeddings - we want to map given input data to the most corresponding Word2Vec label vector. To do this we have to initialize weights of this layer and freeze them.

In [36]:
def map_layer_init(w2c_vectors: list) -> torch.Tensor:
    vectors = np.asarray(w2c_vectors, dtype=float)
    vectors = torch.from_numpy(vectors)
    return vectors[:, -1, :].to(torch.float32)

In [37]:
map_layer = map_layer_init(labels_vectors)
print(f"Size: {map_layer.size()}\nType: {map_layer.dtype}")


Size: torch.Size([16, 100])
Type: torch.float32


## **Neural Network for Network Data**

In [38]:
class NetNet(nn.Module):
    def __init__(self, input_dim: int, words_embeddings_dim: int, output_dim: int, labels_vectors: torch.Tensor):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.linear3 = nn.Linear(256, words_embeddings_dim)
        self.bn3 = nn.BatchNorm1d(words_embeddings_dim)
        self.linear4 = nn.Linear(words_embeddings_dim, output_dim)
        self.ReLU = nn.LeakyReLU()

        # weight initialize
        self.linear4.weight.data = map_layer_init(w2c_vectors=labels_vectors)
        # freeze layer weights
        self.linear4.weight.requires_grad = False

        self.prev_x = torch.empty([])


    def forward(self, x):
        x = self.ReLU(self.linear1(x))
        x = self.ReLU(self.linear2(x))
        x = self.ReLU(self.linear3(x))
        x = self.linear4(x)
        x = F.softmax(x, dim=1)
        return x

In [39]:
def print_model_layer_gradients(model):
    for name, layer in model.named_modules():
        if len(list(layer.named_modules())) == 1 and name != 'ReLU':
            print(f"Layer: {name}\nGradients: {layer.weight.grad}")

### **Training**

In [40]:
def train_model(model: NetNet, epochs: int, data_loader: data.DataLoader, loss_fn: nn.MSELoss, additional_eps=1e-06):
        model.train()
        for epoch in range(epochs):
            for inputs, labels in data_loader:
                if not torch.isfinite(inputs).all():
                      inputs = torch.nan_to_num(inputs) # removing nan values
                if additional_eps:
                      inputs = torch.add(inputs, additional_eps)

                labels = labels.to(device)
                inputs = inputs.to(device)

                outputs = model(inputs)
                loss = loss_fn(outputs, labels.to(torch.long))
                loss.backward(retain_graph=True)
                model.optim.step()
                model.optim.zero_grad()

            #print(f'Labels: {labels}\nOutputs: {outputs}')
            print(f"Epoch: {epoch}, loss: {loss.item():.3}")

In [41]:
model = NetNet(
    input_dim=input_dim,
    words_embeddings_dim=word_vector_size,
    output_dim=output_dim,
    labels_vectors=labels_vectors
)
model.to(device)

NetNet(
  (linear1): Linear(in_features=78, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=100, out_features=16, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
)

In [42]:
learning_rate = 1e-06

In [43]:
loss_fn = nn.NLLLoss()
model.optim = optim.Adam(model.parameters(), lr=learning_rate)
torch.autograd.set_detect_anomaly(True)
train_model(model=model, epochs=5, data_loader=train_data_loader, loss_fn=loss_fn)

Epoch: 0, loss: -0.0777
Epoch: 1, loss: -0.0861
Epoch: 2, loss: -0.0876
Epoch: 3, loss: -0.0931
Epoch: 4, loss: -0.0956


### **Removing last layer**

In [44]:
zsl_model = nn.Sequential(*(list(model.children())[:6] + list(model.children())[7:]))
zsl_model.to(device)

Sequential(
  (0): Linear(in_features=78, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=256, out_features=100, bias=True)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): LeakyReLU(negative_slope=0.01)
)

In [45]:
print(model)
print(zsl_model)

NetNet(
  (linear1): Linear(in_features=78, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=100, out_features=16, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
)
Sequential(
  (0): Linear(in_features=78, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=256, out_features=100, bias=True)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, a

### **Evaluation**
For this step we will calculate euclidean distance and find the vector which is the closest.

In [46]:
def find_closest_vector(vector: torch.Tensor, labels_vectors: torch.Tensor):
    min_dist = float('inf')
    min_dist_label = float('inf')
    pdist = torch.nn.PairwiseDistance(p=2)
    for label, label_vector in enumerate(labels_vectors, start=0):
        #dist = torch.cdist(label_vector, vector, p=2)
        #dist = dist.squeeze(dim=0)
        dist = pdist(label_vector, vector)
        if dist < min_dist:
            min_dist = dist
            min_dist_label = float(label)
    return min_dist_label, min_dist

In [47]:
def evaluate_model(model: NetNet, data_loader: data.DataLoader, labels_vectors: torch.Tensor):
        model.eval()
        true_predictions, predicitons_amount = 0., 0.

        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels_vectors = labels_vectors.to(device)

                pred_input = model(inputs)
                pred_label, dist = find_closest_vector(vector=pred_input[0], labels_vectors=labels_vectors)
                #predictions = predictions.squeeze(dim=1)
                true_predictions += int(pred_label == labels[0])
                predicitons_amount += 1

            accuracy = 100.0 * true_predictions / predicitons_amount

        print(f"Accuracy of the model: {accuracy:4.2f}%")

In [48]:
evaluate_model(zsl_model, test_data_loader, map_layer)

Accuracy of the model: 70.12%
